In [0]:
dbutils.widgets.text("container","data")
dbutils.widgets.text("blob","HE_Regions.zip")
container= dbutils.widgets.get("container")
blob = dbutils.widgets.get("blob")

In [0]:
%pip install uploadshapefile==1.0.0


Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
from pyspark.sql import SparkSession
from datetime import datetime
from uploadshapefile import functions
from pytz import timezone


spark = SparkSession.builder.appName("Shapefile_load").getOrCreate()

upload_result = functions.upload(container,blob)#load data
log_data = [(datetime.now(timezone('Asia/Kolkata')).isoformat(), str(upload_result))]# Prepare data for logging
log_df = spark.createDataFrame(log_data, schema=["timestamp", "upload_result"])# Create a DataFrame

log_df.write.format("delta").mode("append").saveAsTable("shapefile_processing_logs")# Save as a Delta Lake table


In [0]:
if 'error' in str(upload_result):
    status='error'
else:
    status='success'
dbutils.notebook.exit((str(upload_result),status))